In [1]:
import pandas as pd
import numpy as np
import yaml
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
from C_functions_opti import get_LHS_samples, model_optimization, plot_estimation

##### 1. Load all data that is needed

In [2]:
# Load experimental data
df_exp = pd.read_csv('data/data_combined.csv')
biomass_exp = df_exp['Biomass [g/L]']
substrate_exp = df_exp['Glucose [g/L]']

In [3]:
# Load parameters from YAML file
with open('config/parameters.yml', 'r') as file:
    param = yaml.safe_load(file)

##### 2. Define all possible equations for mu and qs

In [4]:
mu0 = lambda mu_max, c_glucose, Ks: mu_max * c_glucose / (c_glucose + Ks) # -- MONOD
mu1 = lambda mu_max, c_biomass, X_max: mu_max * (1 - (c_biomass/ X_max)) # -- LOGISTIC
mu2 = lambda mu_max, c_glucose, Ks, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks)) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC
mu3 = lambda mu_max, c_glucose, Ks, Ki, c_biomass, X_max: mu_max * (c_glucose / (c_glucose + Ks + (c_glucose**2/ Ki))) * (1 - (c_biomass/ X_max)) # -- MONOD + LOGISTIC + INHIBITION

In [5]:
qs0 = lambda qs_max, c_glucose, Ks_qs: qs_max * c_glucose / (Ks_qs + c_glucose) # -- MONOD
qs1 = lambda qs_max, c_glucose, Ks_qs, Ki, glu_met: qs_max * c_glucose / (Ks_qs + c_glucose) * (Ki / (Ki + glu_met)) # -- MONOD + NON COMPETITIVE INHIBITION
qs2 = lambda Yxs, f_glucose, V: 1/Yxs * f_glucose / V # -- YIELD
qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag))) # -- MONOD + METABOLIZED GLU 

In [6]:
mu_all=[mu0, mu1, mu2, mu3]
qs_all=[qs0, qs1, qs2, qs3]

##### 3. Define functions that are needed

In [7]:
# Root mean squared error is the objective function
def objective_function(parameters, mu_eq, num_mu, qs_eq, num_qs):
    # Solve the model using the optimal parameters
    time_pred, biomass_pred, substrate_pred, volume_pred = model_optimization(param, parameters, mu_eq, num_mu, qs_eq, num_qs)  # Solve the model using the current parameters
    biomass = pd.concat([biomass_exp, pd.Series(biomass_pred)], axis=1, keys=['biomass_exp', 'biomass_pred']).dropna()
    biomass_exp_ = biomass['biomass_exp'].values
    biomass_pred_ = biomass['biomass_pred'].values
    mse_x = mean_squared_error(biomass_exp_, biomass_pred_)  # Calculate mean squared error for biomass

    glucose = pd.concat([substrate_exp, pd.Series(substrate_pred)], axis=1, keys=['substrate_exp', 'substrate_pred']).dropna()
    substrate_exp_ = glucose['substrate_exp'].values
    substrate_pred_ = glucose['substrate_pred'].values
    mse_s = mean_squared_error(substrate_exp_, substrate_pred_)  # Calculate mean squared error for substrate
    
    # Calculate the combined rmse
    mse = (mse_x + mse_s)/2
    rmse = np.sqrt(mse)  # Calculate root mean squared error
    return rmse, time_pred, biomass_pred, substrate_pred, volume_pred

##### 4. Define set of parameters in config file

In [8]:
## not needed anymore
# Extract the parameter combination and the number of the set in order to save it in the correct folder
est_mu_max = param['est_mu_max'] # ['set0']
est_Ks = param['est_ks']

In [9]:
# Set the number of samples and parameters
num_samples = 100
num_parameters = 9

# Define the ranges for each parameter
parameter_bounds = [
    [0.01, 0.6],    # Range for parameter 1 mu_max
    [18, 25],       # Range for parameter 2 X_max
    [0.1, 10.0],    # Range for parameter 3 - Ks
    [0.01, 0.1],    # Range for parameter 4 - Ks_qs
    [6.0, 8.0],     # Range for parameter 5 - Ki
    [0.4, 0.5],     # Range for parameter 6 - Yxs
    [0.5, 1.5],     # Range for parameter 7 - qs_max
    [0.05, 0.15],   # Range for parameter 8 - m_s
    [0.001, 0.02],  # Range for parameter 9 - lag
]

In [10]:
LHS_samples = get_LHS_samples(num_samples, num_parameters, parameter_bounds)
LHS_samples.shape

(100, 9)

##### 5. Run it

In [11]:
df_all_sets = pd.DataFrame(columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
for set_num in range(LHS_samples.shape[0]):
    # Save all parameters and equations and the RMSE in a dataframe
    ## with the beginning of one set a new rmse_overview will be created
    rmse_one_set = []
    #key = f'set{set_num}' ; init_p = est_mu_max[key]
    init_p = list(LHS_samples[set_num, :])
    for i in range(len(mu_all)):
        for j in range(len(qs_all)):
            # Define the combination of equations
            mu_eq = mu_all[i]; num_mu = i
            qs_eq = qs_all[j]; num_qs = j

            # Make the predictions and calculate the error
            rmse, time_pred, biomass_pred, substrate_pred, volume_pred = objective_function(init_p, mu_eq, num_mu, qs_eq, num_qs)
            # save the parameters in a dataframe
            append_list=[set_num, i, j, init_p[0], init_p[1], init_p[2], init_p[3], init_p[4], init_p[5], init_p[6], init_p[7], init_p[8], round(rmse, 3)]
            rmse_one_set.append(append_list)

            # Make a plot and save it
            title = f'set{set_num}/ mu{i} - qs{j} - rmse: {round(rmse, 3)}'
            plot_name = f'set{set_num}_mu{i}_qs{j}_rmse{int(rmse)}'

            if rmse <= float(5):
                plot_estimation(time_pred, biomass_pred, substrate_pred, volume_pred, title, plot_name, set_num)

    # save the parameters of one set
    df_1set = pd.DataFrame(rmse_one_set, columns=['set', 'mu', 'qs', 'mu_max', 'X_max', 'Ks', 'Ks_qs', 'Ki', 'Yxs', 'qs_max', 'm_s', 'lag', 'rmse'])
    df_all_sets = pd.concat([df_all_sets,df_1set], axis=0, ignore_index=True)


df_all_sets.sort_values(by=['rmse'], ascending=True, inplace=True)
df_all_sets.to_csv(f'data/estimation/LHS_sampling/data.csv')

df_all_sets.head(10)

/var/folders/nf/38rh41fn3szgbs5wpsdwjvk00000gn/T/ipykernel_56473/1772199970.py:4: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag))) # -- MONOD + METABOLIZED GLU
/var/folders/nf/38rh41fn3szgbs5wpsdwjvk00000gn/T/ipykernel_56473/1772199970.py:4: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag))) # -- MONOD + METABOLIZED GLU
/var/folders/nf/38rh41fn3szgbs5wpsdwjvk00000gn/T/ipykernel_56473/1772199970.py:4: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_glucose, Ks_qs, glu_met, lag: qs_max * c_glucose / (Ks_qs + c_glucose) * (1 / (np.exp(glu_met * lag))) # -- MONOD + METABOLIZED GLU
/var/folders/nf/38rh41fn3szgbs5wpsdwjvk00000gn/T/ipykernel_56473/1772199970.py:4: RuntimeWarning: overflow encountered in exp
  qs3 = lambda qs_max, c_

,set,mu,qs,mu_max,X_max,Ks,Ks_qs,Ki,Yxs,qs_max,m_s,lag,rmse
135,8,1,3,0.148886,19.290590,6.611292,0.044241,7.936618,0.441633,1.153481,0.128394,0.012421,3.997
4,0,1,0,0.151871,22.814298,5.925625,0.098553,6.450598,0.445355,0.979794,0.132838,0.012811,4.014
791,49,1,3,0.184308,20.495291,1.015502,0.091777,7.519081,0.488124,0.782054,0.060107,0.008212,4.032
132,8,1,0,0.148886,19.290590,6.611292,0.044241,7.936618,0.441633,1.153481,0.128394,0.012421,4.469
788,49,1,0,0.184308,20.495291,1.015502,0.091777,7.519081,0.488124,0.782054,0.060107,0.008212,4.580
588,36,3,0,0.436401,21.999172,1.244415,0.096272,7.673315,0.457420,1.334852,0.118667,0.004541,4.940
1224,76,2,0,0.418497,18.383161,3.927158,0.056033,6.497087,0.474755,0.638416,0.139975,0.014267,5.342
299,18,2,3,0.440989,18.804173,2.706261,0.013418,6.432227,0.494693,0.716107,0.099844,0.007946,5.495
1543,96,1,3,0.145176,22.928657,7.925164,0.064290,7.587177,0.433008,1.464007,0.145944,0.017012,5.804
1557,97,1,1,0.232451,19.146513,6.904294,0.052744,6.475590,0.427733,1.256315,0.066871,0.006484,5.936


In [12]:
df_LHS = pd.read_csv('data/estimation/LHS_sampling/data.csv')